# Classificador de imágenes (MINST Database)

## Librerías y clases necesarías

In [ ]:
# Pytorch
import torch
import torchvision

# Tensorboard
from torch.utils.tensorboard import SummaryWriter

# Utils
import matplotlib.pyplot as plt
import os
import numpy as np


### Clases


In [ ]:
class EarlyStopping:
    """Parada temprara del entrenamiento si el coste de validación no mejora tras {patience} iteraciones"""
    def __init__(self, ident, path, patience=7, verbose=True, delta=0):

        self.ident = ident
        self.path = path
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.stop_iter = 0

    def __call__(self, val_loss, model, i):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            #print(f'EarlyStopping counter: {self.counter} de {self.patience}')
            if self.counter >= self.patience:
              if not self.early_stop:
                self.early_stop = True
                self.stop_iter = i - self.patience
                if self.verbose:
                  print(f'[{i}] Mejora estancada por {self.patience} iteraciones ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Activando early stop ...')
              
        else:
          self.save_checkpoint(val_loss, model)
          self.best_score = score
          self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''Guarda el valor de los parámetros en checkpoint'''
        torch.save(model.state_dict(), os.path.join(self.path, f'checkpoint-{self.ident}.pt'))
        self.val_loss_min = val_loss

### Conectar google colab a sistema de archivos en la nube (Google Drive)

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
data_path = os.path.join(os.getcwd(),'gdrive', 'My Drive','Colab Notebooks', 'data', 'mnist')
log_path = os.path.join(os.getcwd(),'gdrive', 'My Drive','Colab Notebooks', 'logs', 'minst')
os.makedirs(data_path, exist_ok=True, )
os.makedirs(log_path, exist_ok=True)
tensorboard_path = f"'{log_path}'"

## Cargar TensorBoard



In [ ]:
!pkill tensorboard
!pkill ngrok

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir $tensorboard_path --host localhost --port 6006

### Utilizar ngrok para ver tensorboard en un navegador

In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

In [ ]:
get_ipython().system_raw('./ngrok http 6006 &')

In [ ]:
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

## Preparando el conjunto de datos

In [ ]:
n_epochs = 15

# Fijando la semilla siempre nos dara los mismo resultados cada vez que lo corramos
random_seed = 1

batch_size_train = 64
validation_set_size = 10000

learning_rate = 0.001

momentum = 0.9
log_interval = 10
patience_value = 3


### Datos de entrenamiento y testeo

#### Manejo de los DataLoaders
> [Enlace a la documentación](https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader)

In [ ]:
data_set =   torchvision.datasets.MNIST(data_path, train=True, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor()
                               # , torchvision.transforms.Normalize((0.1307,), (0.3081,))
                             ]))

test_set = torchvision.datasets.MNIST(data_path, train=False, download = True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor()
                              # , torchvision.transforms.Normalize((0.1307,), (0.3081,))
                             ]))

train_set, val_set = torch.utils.data.random_split(data_set, [len(data_set) - validation_set_size , validation_set_size])
validation_loader = torch.utils.data.DataLoader(val_set, batch_size = validation_set_size, shuffle=False)
no_batch_train_loader = torch.utils.data.DataLoader(train_set, batch_size=len(data_set)-validation_set_size, shuffle=False)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size_train, shuffle=False)
test_loader = torch.utils.data.DataLoader(test_set,batch_size = len(data_set), shuffle=False)

### Cargar imágenes en tensorboard


In [ ]:
SGD_wr = SummaryWriter(os.path.join(log_path, 'images'))

In [ ]:
def load_images(wr, loader, set_name):
  print(f"Saving {set_name}...")
  for idx, (images, labels) in enumerate(loader):
    batch_grid = torchvision.utils.make_grid(images)
    wr.add_image(f"{set_name}/Lote-{idx}", batch_grid)
  print(f"{set_name} saved.")

In [ ]:
load_images(SGD_wr, train_loader, 'Training data')
load_images(SGD_wr, test_loader, 'Test data')
load_images(SGD_wr, validation_loader, 'Validation data')

### Muestra de los datos

In [ ]:
test_examples = enumerate(test_loader)
test_batch_idx, (example_test_data, example_test_targets) = next(test_examples)

train_examples = enumerate(train_loader)
train_batch_idx, (example_train_data, example_train_targets) = next(train_examples)

In [ ]:
print(f"Dimensión de un batch del conjunto de datos de test es {example_test_data.shape}")

In [ ]:
print(f"Dimensión de un batch del conjunto de datos de test es {example_train_data.shape}")

In [ ]:
print(f"Dimensión de los conjuntos:\nEntrenamiento: {len(data_set) - validation_set_size }\nValidación: {validation_set_size}\nTest: {len(test_set)}")

In [ ]:
fig = plt.figure(figsize=(10,5))
for i in range(6):
  plt.subplot(2,3,i+1)
  plt.tight_layout()
  plt.imshow(example_train_data[i][0], cmap='gray', interpolation='none')
  plt.title("Representación: {}".format(example_train_targets[i]), {'fontsize':15})
  plt.xticks([])
  plt.yticks([])
  
fig.show

## Construyendo la red neuronal

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [ ]:
class Net(nn.Module):
    def __init__(self, name):
        super(Net, self).__init__()
        self.name = name
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2(x), 2))
        
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        # x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

### Creación de modelos a comparar fijación de criterios de optimización
> Optimización --> 
- Gradiente descendente estocástico
- Gradiente descendente estocastico con momento
- ADAM

In [ ]:
torch.manual_seed(random_seed)
SGD_modelo = Net("Modelo con gradiente descendente estocástico")
SGD_optimizer = optim.SGD(SGD_modelo.parameters(), lr=learning_rate)
SGD_early_stopper = EarlyStopping('SGD', data_path, patience=patience_value)
SGD_wr = SummaryWriter(os.path.join(log_path,'SGD'))
SGD_val_wr = SummaryWriter(os.path.join(log_path,'SGD_val'))

torch.manual_seed(random_seed)
SGDM_modelo = Net("Modelo con gradiente descendente estocástico con momento")
SGDM_optimizer = optim.SGD(SGDM_modelo.parameters(), lr=learning_rate, momentum=momentum)
SGDM_early_stopper = EarlyStopping('SGD', data_path, patience=patience_value)
SGDM_wr = SummaryWriter(os.path.join(log_path,'SGDM'))
SGDM_val_wr = SummaryWriter(os.path.join(log_path,'SGDM_val')) 


torch.manual_seed(random_seed)
ADAM_modelo = Net("Modelo con ADAM")
ADAM_optimizer = optim.Adam(ADAM_modelo.parameters(), lr=learning_rate)
ADAM_early_stopper = EarlyStopping('ADAM', data_path, patience=patience_value)
ADAM_wr = SummaryWriter(os.path.join(log_path,'ADAM'))
ADAM_val_wr = SummaryWriter(os.path.join(log_path,'ADAM_val'))



In [ ]:
p = sum(p.numel() for p in SGD_modelo.parameters() if p.requires_grad)
print(f'El número de parámetros entrenables es: {p}')

## Transformación del input en el output

In [ ]:
examples_train_data = enumerate(train_loader)
batch_idx, (example_data, example_targets) = next(examples_train_data)

### Salida de bloque uno (Convolucional)

#### Convolución

*   Padding = 0
*   Filters size = 5 x 5
*   Number of features map per image = 10
*   Size of the features map = 24 x 24

En este caso el batch es igual a 64 por lo que existen 64 x 10 mapas de características.

In [ ]:
SGD_modelo.conv1(example_data).shape

#### Pooling

* Kernel size = 2 x 2
* Features map size = 12 x 12

In [ ]:
F.max_pool2d(SGD_modelo.conv1(example_data), 2).shape

#### Función de activación --> ReLu

Solo lo valores mayores que cero permanecen

In [ ]:
block_one_output = F.relu(F.max_pool2d(SGD_modelo.conv1(example_data), 2))
print("La dimensión de cada mapa de características es {} \n ".format(block_one_output[0][0].shape))
block_one_output.shape

### Salida de bloque dos (Convolucional)

#### Convolución

*   Padding = 0
*   Filters size = 5 x 5
*   Number of features map per image = 20
*   Size of the features map = 8 x 8 

En este caso el batch es igual a 64 por lo que existen 64 x 20 mapas de características.

In [ ]:
SGD_modelo.conv2(block_one_output).shape

#### Pooling

* Kernel size = 2 x 2
* Features map size = 4 x 4

In [ ]:
F.max_pool2d(SGD_modelo.conv2(block_one_output), 2).shape

#### Función de activación --> ReLu

Solo lo valores mayores que cero permanecen

In [ ]:
block_two_output = F.relu(F.max_pool2d(SGD_modelo.conv2(block_one_output), 2))
print("La dimensión de cada mapa de características es {} \n ".format(block_two_output[0][0].shape))
block_two_output.shape

### Salida de bloque tres

Transforma los mapas de características en un vector de características

>El argumento -1 indica que el número de imágenes en el batch se infiere

In [ ]:
block_three_output = block_two_output.view(-1, 20 * 4 * 4)
block_three_output.shape

### Salida del bloque cuatro
Se opera como si fuera una Feed Forward Neural Network
> Función de activación ReLu <br>
> Reduce el vector de características de 320 a 50

In [ ]:
block_four_output = F.relu(SGD_modelo.fc1(block_three_output))
block_four_output.shape

### Salida del bloque cinco (Función Softmax)
Se reduce de la salida de la última capa a un vector de 10 valores
> Número de clases = 10 (rango 0-9)

In [ ]:
  block_five_output =  F.log_softmax(SGD_modelo.fc2(block_four_output), dim=1)
  block_five_output.shape

## Entrenando la red neuronal

Función de coste
> $Cross\_entropy = nll\_loss(log\_softmax(x))$

 [Negative log likelihood loss (nll)](https://ljvmiranda921.github.io/notebook/2017/08/13/softmax-and-the-negative-log-likelihood/)

* Función de activación Softmax --> comúnmente utilizada para problemas  de aprendizaje multiclase donde un conjunto de características puede asociaser a 1 de K clases.



In [ ]:
def train(epoch, modelo, optimizer, wr:SummaryWriter):

# train epoch
  for batch_idx, (data, target) in enumerate(train_loader):

    modelo.train()
    optimizer.zero_grad()
    output = modelo(data)
    loss = F.nll_loss(output, target, reduction='mean')
    loss.backward()
    optimizer.step()
    wr.add_scalar('Loss/train', loss.item(), batch_idx + (epoch - 1) * len(train_loader))
    
# end epoch


In [ ]:
def train_error(epoch, modelo, wr):
  modelo.eval()
  correct = 0
  train_loss = 0
  with torch.no_grad():
    for data, target in no_batch_train_loader:
      output = modelo(data)
      train_loss += F.nll_loss(output, target, reduction='mean').item()
    wr.add_scalar('Loss/early', train_loss, epoch)
  print(f'\nTrain set: Loss: {train_loss:.4f}')

In [ ]:
def validation(epoch, modelo, wr, es):
  modelo.eval()
  correct = 0
  validation_loss = 0
  with torch.no_grad():
    for data, target in validation_loader:
      output = modelo(data)
      validation_loss += F.nll_loss(output, target, reduction='mean').item()
      pred = output.data.max(1, keepdim=True)[1]
      correct += pred.eq(target.data.view_as(pred)).sum()
    es(validation_loss, modelo, epoch)
    wr.add_scalar('Loss/early', validation_loss, epoch)
    wr.add_scalar('Accuracy/Validation', 100. * correct / len(validation_loader.dataset), epoch)
  print(f'\nValidation set: Avg. loss: {validation_loss:.4f}, Accuracy: {correct}/{len(validation_loader.dataset)} ({100. * correct / len(validation_loader.dataset):.0f}%)\n')

In [ ]:
def test(epoch, modelo, wr):
  modelo.eval()
  correct = 0
  test_loss = 0
  with torch.no_grad():
    for data, target in test_loader: 
      output = modelo(data)
      test_loss += F.nll_loss(output, target, reduction='mean').item()
      # pred --> Devuelve el index del valor maximo (predicción)
      pred = output.data.max(1, keepdim=True)[1]
      correct += pred.eq(target.data.view_as(pred)).sum()
    wr.add_scalar('Loss/test', test_loss, epoch)
    wr.add_scalar('Accuracy/test', 100. * correct / len(test_loader.dataset), epoch)
  print(f'\nTest set: Avg. loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} ({100. * correct / len(test_loader.dataset):.0f}%)\n\n')

### Rutina de aprendizaje


In [ ]:
print("Training with SGD\n ***************** Sin entrenamiento *********************")
train_error(0, SGD_modelo, SGD_wr)
validation(0, SGD_modelo, SGD_val_wr, SGD_early_stopper)
test(0, SGD_modelo, SGD_wr)

for epoch in range(1, n_epochs + 1):
  print(f'***************** Época: {epoch} *********************')
  train(epoch, SGD_modelo, SGD_optimizer, SGD_wr)
  
  train_error(epoch, SGD_modelo, SGD_wr)
  validation(epoch, SGD_modelo, SGD_val_wr, SGD_early_stopper)
  test(epoch, SGD_modelo, SGD_wr)
  


In [ ]:
print("Training with SGDM\n***************** Sin entrenamiento *********************")
train_error(0, SGDM_modelo, SGDM_wr)
validation(0, SGDM_modelo, SGDM_val_wr, SGDM_early_stopper)
test(0, SGDM_modelo, SGDM_wr)

for epoch in range(1, n_epochs + 1):
  print(f'***************** Época: {epoch} *********************')
  train(epoch, SGDM_modelo, SGDM_optimizer, SGDM_wr)
  train_error(epoch, SGDM_modelo, SGDM_wr)
  validation(epoch, SGDM_modelo, SGDM_val_wr, SGDM_early_stopper)
  test(epoch, SGDM_modelo, SGDM_wr)

In [ ]:
print("Training with ADAM\n***************** Sin entrenamiento *********************")
train_error(0, ADAM_modelo, ADAM_wr)
validation(0, ADAM_modelo, ADAM_val_wr, ADAM_early_stopper)
test(0, ADAM_modelo, ADAM_wr)

for epoch in range(1, n_epochs + 1):
  print(f'***************** Época: {epoch} *********************')
  train(epoch, ADAM_modelo, ADAM_optimizer, ADAM_wr)

  train_error(epoch, ADAM_modelo, ADAM_wr)
  validation(epoch, ADAM_modelo, ADAM_val_wr, ADAM_early_stopper)
  test(epoch, ADAM_modelo, ADAM_wr)

## Evaluando los modelos


In [ ]:
if (SGD_early_stopper.early_stop):
  print(f"Ha habido una parada temprana en el modelo de SGD con un coste de {SGD_early_stopper.val_loss_min}\nÉpoca número:{SGD_early_stopper.stop_iter}\n")
else:
  print(f"No ha habido early stop en SGD, siendo el menor coste: {SGD_early_stopper.val_loss_min}\n")

if (SGDM_early_stopper.early_stop):
  print(f"Ha habido una parada temprana en el modelo de SGD con momento con un coste de {SGDM_early_stopper.val_loss_min}\nÉpoca número:{SGDM_early_stopper.stop_iter}\n")
else:
  print(f"No ha habido early stop en SGD con momento, siendo el menor coste: {SGDM_early_stopper.val_loss_min}\n")

if (ADAM_early_stopper.early_stop):
  print(f"Ha habido una parada temprana en el modelo de ADAM con una precisión de {ADAM_early_stopper.val_loss_min}\nÉpoca número:{ADAM_early_stopper.stop_iter}\n")
else:
  print(f"No ha habido early stop en ADAM, siendo el menor coste: {ADAM_early_stopper.val_loss_min}\n")


## License
MIT License

Copyright (c) 2020 Álvaro Rubio Segovia

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.